In [ ]:
images=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images")
annotations=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations")
train=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/train.csv"))
submission=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/submission.csv"))

**Creating directories for train and validation data**

In [ ]:
!mkdir /kaggle/working/images
!mkdir /kaggle/working/images/train
!mkdir /kaggle/working/images/validation
!mkdir /kaggle/working/images/train/face_with_mask
!mkdir /kaggle/working/images/train/face_no_mask
!mkdir /kaggle/working/images/validation/face_with_mask
!mkdir /kaggle/working/images/validation/face_no_mask

**Selecting only images having class as face mask or face_no_mask and ignoring other classes**

In [ ]:
source="/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/"
train="/kaggle/working/images/train/face_with_mask/"
train_1="/kaggle/working/images/train/face_no_mask/"

test="/kaggle/working/images/validation/face_with_mask/"
test_1="/kaggle/working/images/validation/face_no_mask/"

count=0
with open('/kaggle/input/face-mask-detection-dataset/train.csv') as csvfile:
    readCSV = list(csv.reader(csvfile, delimiter=','))
    print(len([row for row in readCSV[1:] if(row[5]=="face_with_mask" or row[5]=="face_no_mask")]))
    len_train_samples=int(len([row for row in readCSV[1:] if(row[5]=="face_with_mask" or row[5]=="face_no_mask")])*0.7)
    for row in readCSV[1:]:
        if(row[5]=="face_with_mask" or row[5]=="face_no_mask"):
            count+=1
            x1=int(row[1])
            x2=int(row[2])
            y1=int(row[3])
            y2=int(row[4])
            
            image=cv2.imread(src_path+row[0])
            image=image[x2:y2,x1:y1]
            
            if(count<=len_train_samples and row[5]=="face_with_mask"):
                cv2.imwrite(train+str(count)+".jpg",image)
            
            elif(count<=len_train_samples and row[5]=="face_no_mask"):
                cv2.imwrite(train_1+str(count)+".jpg",image)
            
            elif(count>len_train_samples and row[5]=="face_with_mask"):
                cv2.imwrite(test+str(count)+".jpg",image)
            
            elif(count>len_train_samples and row[5]=="face_no_mask"):
                cv2.imwrite(test_1+str(count)+".jpg",image)

**Model framework**

In [ ]:
    model = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

**Image augamentation**

In [ ]:
training_dir = "/kaggle/working/images/train/"
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(training_dir, batch_size=5, class_mode='binary', target_size=(150, 150))

validation_dir = "/kaggle/working/images/validation/"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=5, class_mode='binary',target_size=(150, 150))

**The below model results into approx 90% training and 89% cv accuracy**

In [ ]:
history = model.fit_generator(train_generator, epochs=20, verbose=1, validation_data=validation_generator)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img = mpimg.imread('/content/unzipped/Medical mask/Medical Mask/images/0001.jpg')
imgplot = plt.imshow(img)

**Using mtcnn to detect faces for inputting to the model for predictions** 

In [ ]:
!pip install mtcnn

In [ ]:
from mtcnn import MTCNN

**Test images**

In [ ]:
import os
images_dir='/content/unzipped/Medical mask/Medical Mask/images'
a = os.listdir(images_dir)
a.sort()

In [ ]:
test_images = a[:1800]
len(test_images)

**Detecting faces for text images**

In [ ]:
detector = MTCNN()
test_df = []
for image in test_images:
    img = plt.imread(os.path.join(images_dir, image))
    faces = detector.detect_faces(img)
    test = []
    for face in faces:
        bounding_box = face['box']
        a=bounding_box[0]
        b=bounding_box[1]
        c=bounding_box[2]
        d=bounding_box[3]
        test.append([image, a,b,c,d])
    test_df.append(test)

In [ ]:
test=[]
for i in test_df:
    if len(i)>0:
        if len(i)==1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)

In [ ]:
import pandas as pd
demo=pd.DataFrame(test)
demo.to_csv('/mtcnn',index=False)

In [ ]:
len(test)

In [ ]:
import cv2

In [ ]:
test[:5]

In [ ]:
sub=[]
rest_image=[]
for i in test:
    sub.append(i[0])
for image in test_images:
    if image not in sub:
        rest_image.append(image)

In [ ]:
test_df_=[]
for image in rest_image:
    img=cv2.imread(os.path.join('/content/unzipped/Medical mask/Medical Mask/images',image))
    faces=detector.detect_faces(img)
    test_=[]
    for face in faces:
        bounding_box=face['box']
        test_.append([image,bounding_box])
    test_df_.append(test_)

**Certain images displayed an error while cropping them hence the use of exception handling ignored those images**

In [ ]:
listt=[]
anss=[]
images_names=[]
coordinates=[]
for i in test:
  try:
      for j in i:
          listt.append(j)
      s=listt[0]
      x=listt[1]
      y=listt[2]
      w=listt[3]
      h=listt[4]
      coordinates.append([x,y,x+w,y+h])
      listt.clear()
      img=plt.imread(os.path.join('/content/unzipped/Medical mask/Medical Mask/images',s))
      output=img[y:y+h,x:x+w]
      output=cv2.resize(output,(150,150))
      anss.append(output)
      images_names.append(s)
  except Exception as e:
    print(str(e))

In [ ]:
import numpy as np
ansss=np.array(anss)

**Predictions**

In [ ]:
op=model.predict(ansss)

In [ ]:
oppp=np.round(op)

In [ ]:
opp=pd.DataFrame(data=oppp,columns=['classname'])

In [ ]:
opp.head()

In [ ]:
opp['classname'].value_counts()

In [ ]:
opp.head()

In [ ]:
opp.replace(0.0,'face_no_mask',inplace=True)
opp.replace(1.0,'face_with_mask',inplace=True)

In [ ]:
opp.head()

In [ ]:
opp['classname'].value_counts()

In [ ]:
names=pd.DataFrame(images_names,columns=['name'])

In [ ]:
coordinates=pd.DataFrame(coordinates,columns=['x1','y1','x2','y2'])

In [ ]:
coordinates.head()

In [ ]:
new=pd.DataFrame()

In [ ]:
new['name']=names['name']

In [ ]:
new['x1']=coordinates['x1']

In [ ]:
new['y1']=coordinates['y1']
new['x2']=coordinates['x2']
new['y2']=coordinates['y2']

In [ ]:
new['classname']=opp['classname']

**Inverting the list as per the output csv format requirement**

In [ ]:
new=new.iloc[::-1]

In [ ]:
new.head()

In [ ]:
new.to_csv('/final_submition.csv',index=False)